In [23]:
import time
import random
import threading
import datetime
import tkinter as tk
from tkinter import scrolledtext

class Box:
    def __init__(self, color, name):
        self.color = color
        self.name = name

class Robot:
    def __init__(self, name, next_robot=None):
        self.name = name
        self.box = None
        self.next_robot = next_robot
        self.lock = threading.Lock()

    def transport_box(self, box):
        with self.lock:
            self.box = box
            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            text_area.insert(tk.END, f"{current_time}: {self.name} is transporting a {box.color} box {box.name}\n")
            transport_time = 3 if box.color == 'red' else 5
            for _ in range(transport_time):
                if self.box.color != 'red' and box_queue and box_queue[0].color == 'red':
                    current_time = datetime.datetime.now().strftime('%H:%M:%S')
                    text_area.insert(tk.END, f"{current_time}: {self.name} is dropping a {self.box.color} box to transport a red box {self.box.name}\n")
                    self.box = box_queue.pop(0)
                    text_area.insert(tk.END, f"{current_time}: {self.name} is transporting a {self.box.color} box {self.box.name}\n")
                time.sleep(1)
            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            text_area.insert(tk.END, f"{current_time}: {self.name} has transported a {self.box.color} box {self.box.name}\n")
            self.box = None
            time.sleep(1)
        if self.next_robot:
            threading.Thread(target=self.next_robot.transport_box, args=(box,)).start()

def box_generator(robot):
    global box_queue
    box_queue = []
    box_count = 0
    while box_count < 8:
        color = random.choice(['red', 'blue', 'green'])
        box = Box(color, box_count)
        current_time = datetime.datetime.now().strftime('%H:%M:%S')
        text_area.insert(tk.END, f"{current_time}: A {color} box {box.name} appears \n")
        box_queue.append(box)
        if robot.box is None or (robot.box.color != 'red' and color == 'red'):
            threading.Thread(target=robot.transport_box, args=(box_queue.pop(0),)).start()
            
        box_count += 1
        time.sleep(2)

def start_transportation():
    box_generator_thread = threading.Thread(target=box_generator, args=(robots[0],))
    box_generator_thread.start()

robots = [Robot(f'Robot{i+1}') for i in range(4)]
for i in range(3):
    robots[i].next_robot = robots[i+1]

root = tk.Tk()
text_area = scrolledtext.ScrolledText(root)
text_area.pack()
canvas = tk.Canvas(root, width=500, height=200)
canvas.pack()
start_button = tk.Button(root, text="Start Transportation", command=start_transportation)
start_button.pack()

# Create rectangles for each robot
for i, robot in enumerate(robots):
    robot.rectangle = canvas.create_rectangle(50 * i, 50, 50 * (i + 1), 100, fill="white")

root.mainloop()
